<a href="https://colab.research.google.com/github/MiguelAngel9604/spark_exercice_1/blob/main/Spark_Exercise_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/sravanapisupati/SampleDataSet/blob/main/CleansingCSVData.py
https://www.youtube.com/watch?v=XuEQ_t93Ico

In [ ]:
# Pyspark
!pip install pyspark
# Pyspark stubs
!pip install pyspark-stubs

#Creating Spark session
#I decided to use pyspark and pandas to do all the exercices and advantage of both, I'll be using the same session in all the questions and logic.
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, IntegerType
import pyspark
from pyspark import SparkContext
from pyspark import SparkConf

spark = SparkSession.builder \
    .master("local") \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.executor.memory", "500mb") \
    .appName("Exercise1") \
    .getOrCreate()

In [8]:
import pyspark.sql.functions as f

df1 = spark.read.format("csv").option("header", "true").option("multiline", "true").load("/content/user_data-3 - Hoja 1.csv")
df1.show()

+------------------------------------------------------+
|id,ind,fname,lname,apartment,street,city,country,phone|
+------------------------------------------------------+
|                                           001,FN,John|
|                                           001,LN,Wick|
|                                  001,AD,22,Contine...|
|                                     001,PH,9999999999|
|                                       002,FN,Sherlock|
|                                          002,LN,Homes|
|                                  002,AD,44,Baker S...|
|                                     002,PH,8888888888|
|                                            003,FN,Ted|
|                                          003,LN,Lasso|
|                                  003,AD,AFC,Richmo...|
|                                     003,PH,3336669990|
|                                           004,FN,Star|
|                                           004,LN,Lord|
|                              

In [31]:


df = df1.withColumn("id", f.split(f.col("id,ind,fname,lname,apartment,street,city,country,phone"), ',').getItem(0)) \
       .withColumn('ind', f.split(f.col("id,ind,fname,lname,apartment,street,city,country,phone"), ',').getItem(1)) \
       .withColumn('fname', f.split(f.col("id,ind,fname,lname,apartment,street,city,country,phone"), ',').getItem(2)) \
       .withColumn('lname', f.split(f.col("id,ind,fname,lname,apartment,street,city,country,phone"), ',').getItem(3)) \
       .withColumn('apartment', f.split(f.col("id,ind,fname,lname,apartment,street,city,country,phone"), ',').getItem(4)) \
       .withColumn('street', f.split(f.col("id,ind,fname,lname,apartment,street,city,country,phone"), ',').getItem(5)) \
       .withColumn('city', f.split(f.col("id,ind,fname,lname,apartment,street,city,country,phone"), ',').getItem(6)) \
       .withColumn('country', f.split(f.col("id,ind,fname,lname,apartment,street,city,country,phone"), ',').getItem(7)) \
       .withColumn('phone',f.split(f.col("id,ind,fname,lname,apartment,street,city,country,phone"), ',').getItem(8))  
df_1 = df.drop(col("id,ind,fname,lname,apartment,street,city,country,phone"))
df_1.show()

+---+---+----------+-----------------+-----------+------+----+-------+-----+
| id|ind|     fname|            lname|  apartment|street|city|country|phone|
+---+---+----------+-----------------+-----------+------+----+-------+-----+
|001| FN|      John|             null|       null|  null|null|   null| null|
|001| LN|      Wick|             null|       null|  null|null|   null| null|
|001| AD|        22|Continental Hotel|   New York|   USA|null|   null| null|
|001| PH|9999999999|             null|       null|  null|null|   null| null|
|002| FN|  Sherlock|             null|       null|  null|null|   null| null|
|002| LN|     Homes|             null|       null|  null|null|   null| null|
|002| AD|        44|         Baker St|     London|    UK|null|   null| null|
|002| PH|8888888888|             null|       null|  null|null|   null| null|
|003| FN|       Ted|             null|       null|  null|null|   null| null|
|003| LN|     Lasso|             null|       null|  null|null|   null| null|

In [47]:
df_2 = df_1.select("id",
                   f.when(col("ind") == f.lit("FN"), f.col("fname")).otherwise("null").alias("fname"),
                   f.when(col("ind") == f.lit("LN"), f.col("fname")).otherwise("null").alias("lname"),
                   f.when(col("ind") == f.lit("AD"), 
                          f.concat_ws(",",f.col("fname"), f.col("lname"), f.col("apartment"), f.col("street") )).otherwise("null").alias("address"),
                   f.when(col("ind") == f.lit("PH"), f.col("fname")).otherwise("null").alias("phone"))

df_2.show()

+---+--------+-----+--------------------+----------+
| id|   fname|lname|             address|     phone|
+---+--------+-----+--------------------+----------+
|001|    John| null|                null|      null|
|001|    null| Wick|                null|      null|
|001|    null| null|22,Continental Ho...|      null|
|001|    null| null|                null|9999999999|
|002|Sherlock| null|                null|      null|
|002|    null|Homes|                null|      null|
|002|    null| null|44,Baker St, Lond...|      null|
|002|    null| null|                null|8888888888|
|003|     Ted| null|                null|      null|
|003|    null|Lasso|                null|      null|
|003|    null| null|AFC,Richmond, Lon...|      null|
|003|    null| null|                null|3336669990|
|004|    Star| null|                null|      null|
|004|    null| Lord|                null|      null|
|004|    null| null|                    |      null|
|004|    null| null|                null|     

In [57]:
df_3 = df_2.groupBy("id").agg(
    f.first("fname").alias("fname"),
    f.min("lname").alias("lname"),
    f.min("address").alias("address"),
    f.min("phone").alias("phone")
    )

df_3.show()

+---+--------+------+--------------------+----------+
| id|   fname| lname|             address|     phone|
+---+--------+------+--------------------+----------+
|009|Dundappa|  null|                null|      null|
|006|   Bruce| Wayne|1007,Mountain Dri...|1111111111|
|003|     Ted| Lasso|AFC,Richmond, Lon...|3336669990|
|005|   Clark|      |44,Daily Planet, ...|1234567890|
|008|        |Chavan|2013,Mallesh Paly...|          |
|001|    John|  Wick|22,Continental Ho...|9999999999|
|004|    Star|  Lord|                    |          |
|007|  Sachin|Chavan|2013,Mallesh Paly...|1111111111|
|002|Sherlock| Homes|44,Baker St, Lond...|8888888888|
+---+--------+------+--------------------+----------+



In [62]:
df_4 = df_3.filter(
    (f.col("fname") != "null") & (f.col("lname") != "null")
)

df_4.show()

+---+--------+------+--------------------+----------+
| id|   fname| lname|             address|     phone|
+---+--------+------+--------------------+----------+
|006|   Bruce| Wayne|1007,Mountain Dri...|1111111111|
|003|     Ted| Lasso|AFC,Richmond, Lon...|3336669990|
|005|   Clark|      |44,Daily Planet, ...|1234567890|
|008|        |Chavan|2013,Mallesh Paly...|          |
|001|    John|  Wick|22,Continental Ho...|9999999999|
|004|    Star|  Lord|                    |          |
|007|  Sachin|Chavan|2013,Mallesh Paly...|1111111111|
|002|Sherlock| Homes|44,Baker St, Lond...|8888888888|
+---+--------+------+--------------------+----------+



In [72]:
df_5 = df_4.withColumn("apartment",f.split("address",",").getItem(0)) \
        .withColumn("street",f.split("address",",").getItem(1)) \
        .withColumn("city",f.split("address",",").getItem(2))\
        .withColumn("country",f.split("address",",").getItem(3)) 
        

df_5.show()

+---+--------+------+--------------------+----------+---------+-----------------+-----------+-------+
| id|   fname| lname|             address|     phone|apartment|           street|       city|country|
+---+--------+------+--------------------+----------+---------+-----------------+-----------+-------+
|006|   Bruce| Wayne|1007,Mountain Dri...|1111111111|     1007|   Mountain Drive|     Gotham|    USA|
|003|     Ted| Lasso|AFC,Richmond, Lon...|3336669990|      AFC|         Richmond|     London|     UK|
|005|   Clark|      |44,Daily Planet, ...|1234567890|       44|     Daily Planet| Metropolis|    USA|
|008|        |Chavan|2013,Mallesh Paly...|          |     2013|    Mallesh Palya|  Karnataka|  India|
|001|    John|  Wick|22,Continental Ho...|9999999999|       22|Continental Hotel|   New York|    USA|
|004|    Star|  Lord|                    |          |         |             null|       null|   null|
|007|  Sachin|Chavan|2013,Mallesh Paly...|1111111111|     2013|    Mallesh Palya| 

In [73]:
df5.select("id", "fname", "lname", "apartment", "street", "city", "country", "phone").show(truncate=False)

+---+----------+-----------------+-----------+------+----+-------+-----+
|id |fname     |lname            |apartment  |street|city|country|phone|
+---+----------+-----------------+-----------+------+----+-------+-----+
|001|John      |null             |null       |null  |null|null   |null |
|001|Wick      |null             |null       |null  |null|null   |null |
|001|22        |Continental Hotel| New York  | USA  |null|null   |null |
|001|9999999999|null             |null       |null  |null|null   |null |
|002|Sherlock  |null             |null       |null  |null|null   |null |
|002|Homes     |null             |null       |null  |null|null   |null |
|002|44        |Baker St         | London    | UK   |null|null   |null |
|002|8888888888|null             |null       |null  |null|null   |null |
|003|Ted       |null             |null       |null  |null|null   |null |
|003|Lasso     |null             |null       |null  |null|null   |null |
|003|AFC       |Richmond         | London    | UK  